# Lekce 1

V této lekci projdeme následující témata:
- kumulativní součet hodnot,
- vizualizace složené z více sérií

Ke zpracování dat využijeme moduly  `pandas` a ke tvorbě vizualizací modul `matplotlib` a jeho nadstavbu `seaborn`.

V rámci lekce budeme využívat následující datové soubory:
- [sales_actual.csv](data/sales_actual.csv)
- [sales_plan.csv](data/ales_plan.csv)

## Kumulativní součet

Kumulativní součet (running total) nejčastěji používáme v kombinaci s nějakým časovým údajem. Hodnota kumulativního součtu pro nějaký časový údaj je součtem všech hodnot od začátku časového období až pro daný údaj. Například kumulativní tržby za únor jsou součtem tržeb za leden a za únor, kumulativní součet za březen je součtem tržeb za leden, únor a březen atd.

Podívejme se na následující tabulku, která obsahuje plán tržeb firmy v tis. dolarů za jednotlivé měsíce roků 2021 a 2022.

Protože budeme pracovat s poměrně vysokými čísly, nastavíme formát zobrazení čísla tak, aby používal oddělovač tisíců. Plán je uložený v souboru `data/sales_plan.csv`. Pomocí metod `head()` a `tail()` se podiváme na začátek a konec datového souboru.

### Další možnosti využití

- Bankovní účet: Kumulativní součet hodnoty transakcí udává zůstatek na bankovním účtu.
- Webová stránka: Z data o registracích uživatelů můžeme sledovat celkový počet registrovaných uživatelů.
- Řízení projektů: Srovnání kumulativního součtu odpracovaných hodin pro daný úkol udává, kolik bylo celkem na úkolu odpracováno a z kolika procent by měl být úkol hotový.
- Řízení kvality ve výrobě: Pomocí kumulativního součtu můžeme sledovat celkový počet vadných výrobků v čase.
- Řízení zásob: Pomocí kumulativního součtu můžeme sledovat vývoj množství daného výrobku na skladě.

In [48]:
import pandas

# Načteme data
df_plan = pandas.read_csv("sales_plan.csv")
# Zobrazíme si hlavičku (prvních 5 řádků)
df_plan.head()

,year,month,sales
0,2021,1,12000000.0
1,2021,2,9000000.0
2,2021,3,9000000.0
3,2021,4,11000000.0
4,2021,5,12000000.0


In [49]:
# Zobrazíme "ocas" (posledních 5 řádků)
df_plan.tail()

,year,month,sales
31,2023,8,10000000.0
32,2023,9,9000000.0
33,2023,10,9000000.0
34,2023,11,11000000.0
35,2023,12,11000000.0


Kumulativní součet vytvoříme pomocí metody `cumsum()`.

In [50]:
df_plan["sales_plan_cumsum"] = df_plan["sales"].cumsum()
df_plan

,year,month,sales,sales_plan_cumsum
0,2021,1,12000000.0,12000000.0
1,2021,2,9000000.0,21000000.0
2,2021,3,9000000.0,30000000.0
3,2021,4,11000000.0,41000000.0
4,2021,5,12000000.0,53000000.0
5,2021,6,10000000.0,63000000.0
6,2021,7,12000000.0,75000000.0
7,2021,8,11000000.0,86000000.0
8,2021,9,11000000.0,97000000.0
9,2021,10,8000000.0,105000000.0


U kumulativního součtu někdy požadujeme, aby byl pro určité celky "vynulován", nebo-li aby začínal opět od začátku. V našem případě je to u sloupce `year`. Zařídíme to pomocí metody `groupby()`.

In [51]:
df_plan["sales_plan_cumsum"] = df_plan.groupby("year")["sales"].cumsum()
df_plan

,year,month,sales,sales_plan_cumsum
0,2021,1,12000000.0,12000000.0
1,2021,2,9000000.0,21000000.0
2,2021,3,9000000.0,30000000.0
3,2021,4,11000000.0,41000000.0
4,2021,5,12000000.0,53000000.0
5,2021,6,10000000.0,63000000.0
6,2021,7,12000000.0,75000000.0
7,2021,8,11000000.0,86000000.0
8,2021,9,11000000.0,97000000.0
9,2021,10,8000000.0,105000000.0


### Kumulativní součet s využitím agregace

Plán tržeb může mít firma jako jednoduchou tabulku, údaje o skutečných kontraktech (např. v účetním systému) jsou ale složitější, protože budou zpravidla obsahovat informace o zákazníkovi, datum uzavření kontraktu atd.

In [52]:
df_actual = pandas.read_csv("sales_actual.csv")
df_actual = df_actual.sort_values("date")
df_actual.head()

,date,company,country,contract_value,sales_manager
529,2021-01-04,"Khan, Davidson and Green LLC",United Kingdom,95336.0,Morgan Sarah
272,2021-01-05,"Comencini, Mastroianni e Calarco SPA Group",Italy,425864.0,Jones Leon
341,2021-01-06,Zacco-Callegaro Group s.r.l.,Italy,666505.0,Thomas Denise
538,2021-01-07,Morrison-Johnston PLC,United Kingdom,518377.0,Morgan Sarah
27,2021-01-08,Dionne-Fréchette PLC,Canada,356837.0,Jones Leon


Naším úkolem je porovnat, nakolik se skutečné tržby vyvíjejí podle plánu. K tomu je potřeba provést několik kroků.

1. Provést agregaci podle čísla měsíce a roku.
1. Vypočítat kumulativní součet.
1. Propojit obě tabulky do jedné.

Pořadí kroků je poměrně důležité, protože nesprávné pořadí kroků může vést ke špatným výsledkům. Výsledek můžeme prezentovat například pomocí grafu.

#### Přidání čísla měsíce

Podívejme se na informace o datových typech v naší tabulce.

In [53]:
df_actual.dtypes

date               object
company            object
country            object
contract_value    float64
sales_manager      object
dtype: object

Sloupec s číslem měsíce v tabulce nemáme, můžeme jej ale snadno přidat pomocí vlastnosti `dt.month`.
Nejprve však musíme sloupec `date` převést na typ datum, protože `pandas` ho načetl jako typ `object`.

In [54]:
df_actual["date"] = pandas.to_datetime(df_actual["date"])
df_actual.dtypes

date              datetime64[ns]
company                   object
country                   object
contract_value           float64
sales_manager             object
dtype: object

Nyní můžeme přidat výpočet měsíce. Podobně jako číslo měsíce můžeme přidat i rok.

In [55]:
df_actual["month"] = df_actual["date"].dt.month
df_actual["year"] = df_actual["date"].dt.year
df_actual.head()

,date,company,country,contract_value,sales_manager,month,year
529,2021-01-04,"Khan, Davidson and Green LLC",United Kingdom,95336.0,Morgan Sarah,1,2021
272,2021-01-05,"Comencini, Mastroianni e Calarco SPA Group",Italy,425864.0,Jones Leon,1,2021
341,2021-01-06,Zacco-Callegaro Group s.r.l.,Italy,666505.0,Thomas Denise,1,2021
538,2021-01-07,Morrison-Johnston PLC,United Kingdom,518377.0,Morgan Sarah,1,2021
27,2021-01-08,Dionne-Fréchette PLC,Canada,356837.0,Jones Leon,1,2021


Agregaci již též známe, použijeme k ní metodu `groupby()` a dále metodu `sum()`. Výsledky uložíme do nového dataframe, protože agregace znamená **ztrátu informace** a s původními hodnotami ještě budeme pracovat.

In [56]:
df_actual_grouped = df_actual.groupby(["year", "month"]).sum(numeric_only=True)
df_actual_grouped["sales_actual_cumsum"] = df_actual_grouped.groupby("year")["contract_value"].cumsum()
df_actual_grouped.head()

contract_value  sales_actual_cumsum
year month                                     
2021 1           6649945.0            6649945.0
     2           8091903.0           14741848.0
     3          10722064.0           25463912.0
     4           8550411.0           34014323.0
     5           8786638.0           42800961.0

Nyní můžeme propojit dataframe `df_plan` a `df_actual_grouped` do jednoho dataframe.

In [57]:
df_joined = pandas.merge(df_plan, df_actual_grouped, on=["month", "year"])
df_joined

,year,month,sales,sales_plan_cumsum,contract_value,sales_actual_cumsum
0,2021,1,12000000.0,12000000.0,6649945.0,6649945.0
1,2021,2,9000000.0,21000000.0,8091903.0,14741848.0
2,2021,3,9000000.0,30000000.0,10722064.0,25463912.0
3,2021,4,11000000.0,41000000.0,8550411.0,34014323.0
4,2021,5,12000000.0,53000000.0,8786638.0,42800961.0
5,2021,6,10000000.0,63000000.0,9466508.0,52267469.0
6,2021,7,12000000.0,75000000.0,7754752.0,60022221.0
7,2021,8,11000000.0,86000000.0,8083338.0,68105559.0
8,2021,9,11000000.0,97000000.0,7114588.0,75220147.0
9,2021,10,8000000.0,105000000.0,8300097.0,83520244.0


# Cvičení

Uvažuj, že provádíš analýzu stavu zásob v kamenném obchodě. Obchod je otevřený každý všední den, zásoby jsou přivezeny vždy první pracovní den v měsíci. Pro zjednodušení máme záznamy pouze o třech produktech. Data najdeš v souboru [inventory.csv](data/inventory.csv).

- Pomocí kumulativního součtu vytvoř v tabulce sloupec, který udává množství každého z produktů na skladě.
- V daném období zažíval obchod problémy se zásobováním a některé zboží tak bylo vyprodané. Zjisti pro každý z produktů, kolik dní v roce byl vyprodaný, tj. kolik dní v roce byl stav zásob 0.
- Vytvoř novou tabulku, kde ponecháš pro dané zboží pouze dny, kdy zboží nebylo vyprodáno. Z této tabulky pak spočítej průměrný počet prodaných kusů zboží za den. (Jde nám o to, jaký byl zájem zákazníků o zboží. Dny, kdy bylo zboží vyprodáno, tedy z analýzy vynecháme.)
- Pomocí agregace spočítej, kolik bylo v průměru prodáno každého zboží.


In [58]:
df = pandas.read_csv("data/inventory.csv")
df = df.drop(columns="Unnamed: 0")
df.to_csv("data/inventory.csv", index=False)